In [8]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import json
import time
from tqdm.notebook import tqdm 
import numpy as np

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [9]:
fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geocoded/geo_nm.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1","Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1","Unnamed: 0.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1"], axis =1) 
fips_check

,Unnamed: 0,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,593094,593094,593094,593094,593094,9539388701,4/9/21,682,PPP,PRESBYTERIAN MEDICAL SERVICES,...,999 Other,6214 Outpatient Care Centers,6214.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,500 or more,"1422 Paseo de Peralta, Santa Fe, NM",35.681498,-105.944481,NaN
1,593095,593095,593095,593095,593095,6755517109,4/14/20,682,PPP,"BAM! PIZZA MANAGEMENT, INC.",...,999 Other,7225 Restaurants and Other Eating Places,7225.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,500 or more,"1114 SOUTH UNION AVE, ROSWELL, NM",33.380083,-104.536235,NaN
2,593096,593096,593096,593096,593096,8818167010,4/8/20,682,PPP,"DREAMSTYLE REMODELING, INC.",...,999 Other,2361 Residential Building Construction,2361.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,250 to 499,"1460 N RENAISSANCE BLVD NE, ALBUQUERQUE, NM",35.137004,-106.611125,NaN
3,593097,593097,593097,593097,593097,9499057008,4/9/20,682,PPP,"TLC COMPANY, INC.",...,999 Other,5313 Activities Related to Real Estate,5313.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,250 to 499,"5000 EDITH BLVD, NE, ALBUQUERQUE, NM",35.130454,-106.629499,NaN
4,593098,593098,593098,593098,593098,9341497207,4/28/20,682,PPP,"B&D INDUSTRIES, INC.",...,999 Other,2382 Building Equipment Contractors,2382.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,250 to 499,"9720 Bell Ave SE, Albuquerque, NM",35.069656,-106.539648,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4480,597574,597574,597574,597574,597574,8805347202,4/28/20,682,PPP,SUN RISE ROOFING LLC,...,999 Other,2362 Nonresidential Building Construction,2362.0,NEW MEXICO DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"210 Constituition Ave NW, Albuquerque, NM",35.098598,-106.646976,NaN
4481,597575,597575,597575,597575,597575,9394038307,1/30/21,682,PPS,PICKERING ENTERPRISES LLC,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,NEW MEXICO DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"407 E Broadway, Farmington, NM",36.727896,-108.201310,NaN
4482,597576,597576,597576,597576,597576,9614607003,4/9/20,682,PPP,"FRY COOKS, INC",...,999 Other,7225 Restaurants and Other Eating Places,7225.0,NEW MEXICO DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"2600 21ST ST, CLOVIS, NM",34.419929,-103.177147,NaN
4483,597577,597577,597577,597577,597577,9653187305,5/2/20,682,PPP,MARKS PAINTING MAINTENANCE,...,999 Other,2383 Building Finishing Contractors,2383.0,NEW MEXICO DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"11423 COSTA DEL SOL NE, ALBUQUERQUE, NM",35.159075,-106.511962,NaN


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [4]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #3

,LoanNumber,full_add,Lat,Long
3339,3006758400,"3316 E Historic Highway 66, Gallup, NM",NaN,NaN
3443,8029977308,"100 FRONTERA BLVD, JOGUES, NM",NaN,NaN
4243,5019727208,"4901 Edith Blvd NE, SANDIA PARK, NM",NaN,NaN
4391,2328867106,"1510 Latigo Circle, ROSWELL, NM",NaN,NaN


In [5]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #15866

,LoanNumber,full_add,Lat,Long
0,9539388701,"1422 Paseo de Peralta, Santa Fe, NM",35.681498,-105.944481
1,6755517109,"1114 SOUTH UNION AVE, ROSWELL, NM",33.380083,-104.536235
2,8818167010,"1460 N RENAISSANCE BLVD NE, ALBUQUERQUE, NM",35.137004,-106.611125
3,9499057008,"5000 EDITH BLVD, NE, ALBUQUERQUE, NM",35.130454,-106.629499
4,9341497207,"9720 Bell Ave SE, Albuquerque, NM",35.069656,-106.539648


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [10]:
fips_check.to_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/NM_fips_scraped.csv")


In [11]:
fips_check_scraped = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/NM_fips_scraped.csv") 
fips_check_scraped.head() 

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,593094,593094,593094,593094,593094,9539388701,4/9/21,682,PPP,...,999 Other,6214 Outpatient Care Centers,6214.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,500 or more,"1422 Paseo de Peralta, Santa Fe, NM",35.681498,-105.944481,NaN
1,1,593095,593095,593095,593095,593095,6755517109,4/14/20,682,PPP,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,500 or more,"1114 SOUTH UNION AVE, ROSWELL, NM",33.380083,-104.536235,NaN
2,2,593096,593096,593096,593096,593096,8818167010,4/8/20,682,PPP,...,999 Other,2361 Residential Building Construction,2361.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,250 to 499,"1460 N RENAISSANCE BLVD NE, ALBUQUERQUE, NM",35.137004,-106.611125,NaN
3,3,593097,593097,593097,593097,593097,9499057008,4/9/20,682,PPP,...,999 Other,5313 Activities Related to Real Estate,5313.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,250 to 499,"5000 EDITH BLVD, NE, ALBUQUERQUE, NM",35.130454,-106.629499,NaN
4,4,593098,593098,593098,593098,593098,9341497207,4/28/20,682,PPP,...,999 Other,2382 Building Equipment Contractors,2382.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,250 to 499,"9720 Bell Ave SE, Albuquerque, NM",35.069656,-106.539648,NaN


In [12]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [13]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [14]:
fips_check_scraped['Lat'].replace('None', np.nan, inplace=True)
fips_check_scraped['Long'].replace('None', np.nan, inplace=True)

In [15]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #15866

,LoanNumber,full_add,Lat,Long,FIPS_z
0,9539388701,"1422 Paseo de Peralta, Santa Fe, NM",35.681498,-105.944481,NaN
1,6755517109,"1114 SOUTH UNION AVE, ROSWELL, NM",33.380083,-104.536235,NaN
2,8818167010,"1460 N RENAISSANCE BLVD NE, ALBUQUERQUE, NM",35.137004,-106.611125,NaN
3,9499057008,"5000 EDITH BLVD, NE, ALBUQUERQUE, NM",35.130454,-106.629499,NaN
4,9341497207,"9720 Bell Ave SE, Albuquerque, NM",35.069656,-106.539648,NaN
...,...,...,...,...,...
4480,8805347202,"210 Constituition Ave NW, Albuquerque, NM",35.098598,-106.646976,NaN
4481,9394038307,"407 E Broadway, Farmington, NM",36.727896,-108.201310,NaN
4482,9614607003,"2600 21ST ST, CLOVIS, NM",34.419929,-103.177147,NaN
4483,9653187305,"11423 COSTA DEL SOL NE, ALBUQUERQUE, NM",35.159075,-106.511962,NaN


In [16]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null.count() 

LoanNumber    4481
full_add      4481
Lat           4481
Long          4481
FIPS_z           0
dtype: int64

<h4>FIPS Script

In [17]:
#if geocode file
#lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
#long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [18]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
service = {'service': Service(ChromeDriverManager().install())}
driver = Browser('chrome', **service, headless=True)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('35')] #NM
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/102.0.5005.61/chromedriver_mac64.zip
Driver has been saved in cache [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/102.0.5005.61]


  0%|          | 0/4481 [00:00<?, ?it/s]

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [19]:
geo_id_list =geo_id 

Length Check

In [20]:
print(len(geo_id_list))

4481


Matching FIPS to DataFrame

In [21]:
#for geocode file
#fips_fill = fips_check_notna
#if FIPS available
fips_fill = fips_check_scraped_null

In [22]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
0,9539388701,"1422 Paseo de Peralta, Santa Fe, NM",35.681498,-105.944481,[350490004001062]
1,6755517109,"1114 SOUTH UNION AVE, ROSWELL, NM",33.380083,-104.536235,[350050002013012]
2,8818167010,"1460 N RENAISSANCE BLVD NE, ALBUQUERQUE, NM",35.137004,-106.611125,[350010037451033]
3,9499057008,"5000 EDITH BLVD, NE, ALBUQUERQUE, NM",35.130454,-106.629499,[350010032011003]
4,9341497207,"9720 Bell Ave SE, Albuquerque, NM",35.069656,-106.539648,[350010007151025]
...,...,...,...,...,...
4480,8805347202,"210 Constituition Ave NW, Albuquerque, NM",35.098598,-106.646976,[350010027001044]
4481,9394038307,"407 E Broadway, Farmington, NM",36.727896,-108.201310,[350450004021007]
4482,9614607003,"2600 21ST ST, CLOVIS, NM",34.419929,-103.177147,[350090003081035]
4483,9653187305,"11423 COSTA DEL SOL NE, ALBUQUERQUE, NM",35.159075,-106.511962,[350010037261003]


Remove brackets

In [23]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
0,9539388701,"1422 Paseo de Peralta, Santa Fe, NM",35.681498,-105.944481,350490004001062
1,6755517109,"1114 SOUTH UNION AVE, ROSWELL, NM",33.380083,-104.536235,350050002013012
2,8818167010,"1460 N RENAISSANCE BLVD NE, ALBUQUERQUE, NM",35.137004,-106.611125,350010037451033
3,9499057008,"5000 EDITH BLVD, NE, ALBUQUERQUE, NM",35.130454,-106.629499,350010032011003
4,9341497207,"9720 Bell Ave SE, Albuquerque, NM",35.069656,-106.539648,350010007151025
...,...,...,...,...,...
4480,8805347202,"210 Constituition Ave NW, Albuquerque, NM",35.098598,-106.646976,350010027001044
4481,9394038307,"407 E Broadway, Farmington, NM",36.727896,-108.201310,350450004021007
4482,9614607003,"2600 21ST ST, CLOVIS, NM",34.419929,-103.177147,350090003081035
4483,9653187305,"11423 COSTA DEL SOL NE, ALBUQUERQUE, NM",35.159075,-106.511962,350010037261003


In [24]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #24

,LoanNumber,full_add,Lat,Long,FIPS_z
94,5798917705,"50 LOS BANOS DR, OJO CALIENTE, NM",21.878201,-102.673121,NaN
164,9950078310,"50 Los Banos Dr, Ojo Caliente, NM",21.878201,-102.673121,NaN
756,3817587100,"103 MAGUEY CT, SUNLAND PARK, NM",31.833009,-106.536423,NaN
1380,9511588504,"761 ST CLOUD RD, WINSTON, NM",34.086180,-118.441194,NaN
1560,1930747201,"HWY 84 CR 0324 #14, TIERRA AMARILLA, NM",34.380289,-103.005229,NaN


Parse for Merge

In [25]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
0,9539388701,350490004001062
1,6755517109,350050002013012
2,8818167010,350010037451033
3,9499057008,350010032011003
4,9341497207,350010007151025
...,...,...
4480,8805347202,350010027001044
4481,9394038307,350450004021007
4482,9614607003,350090003081035
4483,9653187305,350010037261003


Read in base data

In [26]:
#if from geocode file
#fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geocoded/geo_nm.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1", "Unnamed: 0.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1"], axis =1) 
#fips_check

#if from FIPS file
fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/NM_fips_scraped.csv") 
#fips_check = fips_check.drop([ "Unnamed: 0.1.1.1.1.1.1.1"], axis =1) 
fips_check.head() 


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,593094,593094,593094,593094,593094,9539388701,4/9/21,682,PPP,...,999 Other,6214 Outpatient Care Centers,6214.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,500 or more,"1422 Paseo de Peralta, Santa Fe, NM",35.681498,-105.944481,NaN
1,1,593095,593095,593095,593095,593095,6755517109,4/14/20,682,PPP,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,500 or more,"1114 SOUTH UNION AVE, ROSWELL, NM",33.380083,-104.536235,NaN
2,2,593096,593096,593096,593096,593096,8818167010,4/8/20,682,PPP,...,999 Other,2361 Residential Building Construction,2361.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,250 to 499,"1460 N RENAISSANCE BLVD NE, ALBUQUERQUE, NM",35.137004,-106.611125,NaN
3,3,593097,593097,593097,593097,593097,9499057008,4/9/20,682,PPP,...,999 Other,5313 Activities Related to Real Estate,5313.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,250 to 499,"5000 EDITH BLVD, NE, ALBUQUERQUE, NM",35.130454,-106.629499,NaN
4,4,593098,593098,593098,593098,593098,9341497207,4/28/20,682,PPP,...,999 Other,2382 Building Equipment Contractors,2382.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,250 to 499,"9720 Bell Ave SE, Albuquerque, NM",35.069656,-106.539648,NaN


Merge with MainFrame

In [27]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,...,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,0,593094,593094,593094,593094,593094,9539388701,4/9/21,682,PPP,...,6214 Outpatient Care Centers,6214.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,500 or more,"1422 Paseo de Peralta, Santa Fe, NM",35.681498,-105.944481,NaN,350490004001062
1,1,593095,593095,593095,593095,593095,6755517109,4/14/20,682,PPP,...,7225 Restaurants and Other Eating Places,7225.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,500 or more,"1114 SOUTH UNION AVE, ROSWELL, NM",33.380083,-104.536235,NaN,350050002013012
2,2,593096,593096,593096,593096,593096,8818167010,4/8/20,682,PPP,...,2361 Residential Building Construction,2361.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,250 to 499,"1460 N RENAISSANCE BLVD NE, ALBUQUERQUE, NM",35.137004,-106.611125,NaN,350010037451033
3,3,593097,593097,593097,593097,593097,9499057008,4/9/20,682,PPP,...,5313 Activities Related to Real Estate,5313.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,250 to 499,"5000 EDITH BLVD, NE, ALBUQUERQUE, NM",35.130454,-106.629499,NaN,350010032011003
4,4,593098,593098,593098,593098,593098,9341497207,4/28/20,682,PPP,...,2382 Building Equipment Contractors,2382.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,250 to 499,"9720 Bell Ave SE, Albuquerque, NM",35.069656,-106.539648,NaN,350010007151025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4480,4480,597574,597574,597574,597574,597574,8805347202,4/28/20,682,PPP,...,2362 Nonresidential Building Construction,2362.0,NEW MEXICO DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"210 Constituition Ave NW, Albuquerque, NM",35.098598,-106.646976,NaN,350010027001044
4481,4481,597575,597575,597575,597575,597575,9394038307,1/30/21,682,PPS,...,5617 Services to Buildings and Dwellings,5617.0,NEW MEXICO DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"407 E Broadway, Farmington, NM",36.727896,-108.201310,NaN,350450004021007
4482,4482,597576,597576,597576,597576,597576,9614607003,4/9/20,682,PPP,...,7225 Restaurants and Other Eating Places,7225.0,NEW MEXICO DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"2600 21ST ST, CLOVIS, NM",34.419929,-103.177147,NaN,350090003081035
4483,4483,597577,597577,597577,597577,597577,9653187305,5/2/20,682,PPP,...,2383 Building Finishing Contractors,2383.0,NEW MEXICO DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"11423 COSTA DEL SOL NE, ALBUQUERQUE, NM",35.159075,-106.511962,NaN,350010037261003


<b>Fill FIPS Columns - if FIPS file

In [28]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,0,593094,593094,593094,593094,593094,9539388701,4/9/21,682,PPP,...,6214.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,500 or more,"1422 Paseo de Peralta, Santa Fe, NM",35.681498,-105.944481,NaN,350490004001062,350490004001062
1,1,593095,593095,593095,593095,593095,6755517109,4/14/20,682,PPP,...,7225.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,500 or more,"1114 SOUTH UNION AVE, ROSWELL, NM",33.380083,-104.536235,NaN,350050002013012,350050002013012
2,2,593096,593096,593096,593096,593096,8818167010,4/8/20,682,PPP,...,2361.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,250 to 499,"1460 N RENAISSANCE BLVD NE, ALBUQUERQUE, NM",35.137004,-106.611125,NaN,350010037451033,350010037451033
3,3,593097,593097,593097,593097,593097,9499057008,4/9/20,682,PPP,...,5313.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,250 to 499,"5000 EDITH BLVD, NE, ALBUQUERQUE, NM",35.130454,-106.629499,NaN,350010032011003,350010032011003
4,4,593098,593098,593098,593098,593098,9341497207,4/28/20,682,PPP,...,2382.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,250 to 499,"9720 Bell Ave SE, Albuquerque, NM",35.069656,-106.539648,NaN,350010007151025,350010007151025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4480,4480,597574,597574,597574,597574,597574,8805347202,4/28/20,682,PPP,...,2362.0,NEW MEXICO DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"210 Constituition Ave NW, Albuquerque, NM",35.098598,-106.646976,NaN,350010027001044,350010027001044
4481,4481,597575,597575,597575,597575,597575,9394038307,1/30/21,682,PPS,...,5617.0,NEW MEXICO DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"407 E Broadway, Farmington, NM",36.727896,-108.201310,NaN,350450004021007,350450004021007
4482,4482,597576,597576,597576,597576,597576,9614607003,4/9/20,682,PPP,...,7225.0,NEW MEXICO DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"2600 21ST ST, CLOVIS, NM",34.419929,-103.177147,NaN,350090003081035,350090003081035
4483,4483,597577,597577,597577,597577,597577,9653187305,5/2/20,682,PPP,...,2383.0,NEW MEXICO DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"11423 COSTA DEL SOL NE, ALBUQUERQUE, NM",35.159075,-106.511962,NaN,350010037261003,350010037261003


Drop Extra FIPS Columns

In [29]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,593094,593094,593094,593094,593094,9539388701,4/9/21,682,PPP,...,999 Other,6214 Outpatient Care Centers,6214.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,500 or more,"1422 Paseo de Peralta, Santa Fe, NM",35.681498,-105.944481,350490004001062
1,1,593095,593095,593095,593095,593095,6755517109,4/14/20,682,PPP,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,500 or more,"1114 SOUTH UNION AVE, ROSWELL, NM",33.380083,-104.536235,350050002013012
2,2,593096,593096,593096,593096,593096,8818167010,4/8/20,682,PPP,...,999 Other,2361 Residential Building Construction,2361.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,250 to 499,"1460 N RENAISSANCE BLVD NE, ALBUQUERQUE, NM",35.137004,-106.611125,350010037451033
3,3,593097,593097,593097,593097,593097,9499057008,4/9/20,682,PPP,...,999 Other,5313 Activities Related to Real Estate,5313.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,250 to 499,"5000 EDITH BLVD, NE, ALBUQUERQUE, NM",35.130454,-106.629499,350010032011003
4,4,593098,593098,593098,593098,593098,9341497207,4/28/20,682,PPP,...,999 Other,2382 Building Equipment Contractors,2382.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,250 to 499,"9720 Bell Ave SE, Albuquerque, NM",35.069656,-106.539648,350010007151025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4480,4480,597574,597574,597574,597574,597574,8805347202,4/28/20,682,PPP,...,999 Other,2362 Nonresidential Building Construction,2362.0,NEW MEXICO DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"210 Constituition Ave NW, Albuquerque, NM",35.098598,-106.646976,350010027001044
4481,4481,597575,597575,597575,597575,597575,9394038307,1/30/21,682,PPS,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,NEW MEXICO DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"407 E Broadway, Farmington, NM",36.727896,-108.201310,350450004021007
4482,4482,597576,597576,597576,597576,597576,9614607003,4/9/20,682,PPP,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,NEW MEXICO DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"2600 21ST ST, CLOVIS, NM",34.419929,-103.177147,350090003081035
4483,4483,597577,597577,597577,597577,597577,9653187305,5/2/20,682,PPP,...,999 Other,2383 Building Finishing Contractors,2383.0,NEW MEXICO DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"11423 COSTA DEL SOL NE, ALBUQUERQUE, NM",35.159075,-106.511962,350010037261003


In [31]:
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1","Unnamed: 0.1.1.1.1.1.1.1","Unnamed: 0.1.1.1.1.1.1.1.1","Unnamed: 0.1.1.1.1.1.1.1.1.1","Unnamed: 0.1.1.1.1.1.1.1.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #27 , notna #22799 / 22826

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
94,94,5798917705,5/1/20,682,PPP,OJO CALIENTE HOLDINGS INC,50 LOS BANOS DR,OJO CALIENTE,NM,87549.0,...,NaN,NaN,NaN,NEW MEXICO DISTRICT OFFICE,d $2-5 million,100 to 249,"50 LOS BANOS DR, OJO CALIENTE, NM",21.878201,-102.673121,NaN
164,164,9950078310,1/31/21,682,PPS,OJO CALIENTE HOLDINGS INC,50 Los Banos Dr,Ojo Caliente,NM,87549.0,...,999 Other,7139 Other Amusement and Recreation Industries,7139.0,NEW MEXICO DISTRICT OFFICE,c $1-2 million,100 to 249,"50 Los Banos Dr, Ojo Caliente, NM",21.878201,-102.673121,NaN
756,756,3817587100,4/12/20,682,PPP,"C&E INDUSTRIAL SERVICES, INC.",103 MAGUEY CT,SUNLAND PARK,NM,88063.0,...,999 Other,8113 Commercial and Industrial Machinery and E...,8113.0,NEW MEXICO DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"103 MAGUEY CT, SUNLAND PARK, NM",31.833009,-106.536423,NaN
1380,1380,9511588504,3/12/21,682,PPS,ST CLOUD MINING COMPANY,761 ST CLOUD RD,WINSTON,NM,87943.0,...,999 Other,2123 Nonmetallic Mineral Mining and Quarrying,2123.0,NEW MEXICO DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"761 ST CLOUD RD, WINSTON, NM",34.086180,-118.441194,NaN
1560,1560,1930747201,4/15/20,682,PPP,LA CLINICA DEL PUEBLO,HWY 84 CR 0324 #14,TIERRA AMARILLA,NM,87575.0,...,999 Other,6214 Outpatient Care Centers,6214.0,NEW MEXICO DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"HWY 84 CR 0324 #14, TIERRA AMARILLA, NM",34.380289,-103.005229,NaN
3339,3339,3006758400,2/4/21,682,PPS,SHIMA INC,3316 E Historic Highway 66,Gallup,NM,87301.0,...,999 Other,4859 Other Transit and Ground Passenger Transp...,4859.0,NEW MEXICO DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"3316 E Historic Highway 66, Gallup, NM",NaN,NaN,NaN
3443,3443,8029977308,5/1/20,682,PPP,SANTA TERESA INTERNATIONAL EXPORT/IMPORT,100 FRONTERA BLVD,JOGUES,NM,88008.0,...,999 Other,1121 Cattle Ranching and Farming,1121.0,NEW MEXICO DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"100 FRONTERA BLVD, JOGUES, NM",NaN,NaN,NaN
4243,4243,5019727208,4/27/20,682,PPP,SIGNPLEX LLC,4901 Edith Blvd NE,SANDIA PARK,NM,87047.0,...,999 Other,5414 Specialized Design Services,5414.0,NEW MEXICO DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"4901 Edith Blvd NE, SANDIA PARK, NM",NaN,NaN,NaN
4391,4391,2328867106,4/10/20,682,PPP,"NEW MEXICO SUBWAY, INC.",1510 Latigo Circle,ROSWELL,NM,88201.0,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,NEW MEXICO DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1510 Latigo Circle, ROSWELL, NM",NaN,NaN,NaN


Convert to File

In [32]:
fips_NM = fips_merge_drop


In [33]:
fips_NM.to_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/NM_fips_scraped.csv") 

Review & Compare

In [34]:
fips_NM = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/NM_fips_scraped.csv") 
fips_NM

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,9539388701,4/9/21,682,PPP,PRESBYTERIAN MEDICAL SERVICES,1422 Paseo de Peralta,Santa Fe,NM,...,999 Other,6214 Outpatient Care Centers,6214.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,500 or more,"1422 Paseo de Peralta, Santa Fe, NM",35.681498,-105.944481,3.504900e+14
1,1,1,6755517109,4/14/20,682,PPP,"BAM! PIZZA MANAGEMENT, INC.",1114 SOUTH UNION AVE,ROSWELL,NM,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,500 or more,"1114 SOUTH UNION AVE, ROSWELL, NM",33.380083,-104.536235,3.500500e+14
2,2,2,8818167010,4/8/20,682,PPP,"DREAMSTYLE REMODELING, INC.",1460 N RENAISSANCE BLVD NE,ALBUQUERQUE,NM,...,999 Other,2361 Residential Building Construction,2361.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,250 to 499,"1460 N RENAISSANCE BLVD NE, ALBUQUERQUE, NM",35.137004,-106.611125,3.500100e+14
3,3,3,9499057008,4/9/20,682,PPP,"TLC COMPANY, INC.","5000 EDITH BLVD, NE",ALBUQUERQUE,NM,...,999 Other,5313 Activities Related to Real Estate,5313.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,250 to 499,"5000 EDITH BLVD, NE, ALBUQUERQUE, NM",35.130454,-106.629499,3.500100e+14
4,4,4,9341497207,4/28/20,682,PPP,"B&D INDUSTRIES, INC.",9720 Bell Ave SE,Albuquerque,NM,...,999 Other,2382 Building Equipment Contractors,2382.0,NEW MEXICO DISTRICT OFFICE,e $5-10 million,250 to 499,"9720 Bell Ave SE, Albuquerque, NM",35.069656,-106.539648,3.500100e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4480,4480,4480,8805347202,4/28/20,682,PPP,SUN RISE ROOFING LLC,210 Constituition Ave NW,Albuquerque,NM,...,999 Other,2362 Nonresidential Building Construction,2362.0,NEW MEXICO DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"210 Constituition Ave NW, Albuquerque, NM",35.098598,-106.646976,3.500100e+14
4481,4481,4481,9394038307,1/30/21,682,PPS,PICKERING ENTERPRISES LLC,407 E Broadway,Farmington,NM,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,NEW MEXICO DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"407 E Broadway, Farmington, NM",36.727896,-108.201310,3.504500e+14
4482,4482,4482,9614607003,4/9/20,682,PPP,"FRY COOKS, INC",2600 21ST ST,CLOVIS,NM,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,NEW MEXICO DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"2600 21ST ST, CLOVIS, NM",34.419929,-103.177147,3.500900e+14
4483,4483,4483,9653187305,5/2/20,682,PPP,MARKS PAINTING MAINTENANCE,11423 COSTA DEL SOL NE,ALBUQUERQUE,NM,...,999 Other,2383 Building Finishing Contractors,2383.0,NEW MEXICO DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"11423 COSTA DEL SOL NE, ALBUQUERQUE, NM",35.159075,-106.511962,3.500100e+14
